# Analytics Backed Business Decisions
#### In this project we will be working with a database called Chinook. This database is associated with a digital music shop. It contains information about employees, customers, purchases, and everything to do with the tracks that are on the platform (artists, albums, genres, playlists, etc.). The following is the schema for this Chinook database.



### The goal of this project is to answer business questions to help Chinook make more informed decisions.

In [7]:
# Import necessary libraries
import sqlite3
import plotly.express as px, plotly.graph_objects as go
import pandas as pd

In [8]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

### Exploring the database tables

In [9]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [12]:
%%sql
SELECT * FROM customer
LIMIT 3

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


## Most Popular Genres

Chinook has recently signed a deal with a record label that specializes in artists from the USA. Furthermore, Chinook is looking to add three albums out of four potential new artists. The following are the artists and their genres. 

**Artist Name** | **Genre**
:---: | :---:
Regal | Hip-Hop
Red Tone | Punk
Meteor and the Girls | Pop
Slim Jim Bites | Blues

In order to determine which three genres should be added to the store the most popular genres need to be determined. To achieve this we will find the genres that sell the most tracks in USA. 

In [30]:
%%sql
WITH 
    tracks_usa AS
    (
        SELECT 
        t.track_id track_id,
        g.name genre,
        c.country country   

        FROM invoice i
        LEFT JOIN customer c ON c.customer_id = i.customer_id
        LEFT JOIN invoice_line il ON il.invoice_id = i.invoice_id
        LEFT JOIN track t ON il.track_id = t.track_id
        LEFT JOIN genre g ON t.genre_id = g.genre_id
        WHERE country = "USA"
    )
    
SELECT
    tu.genre genre,
    COUNT(tu.track_id) number_of_tracks,
    CAST(COUNT(tu.track_id) AS FLOAT) / (SELECT COUNT(*) 
                                             FROM tracks_usa) AS percentage_sold
    FROM tracks_usa tu
    GROUP BY genre ORDER BY number_of_tracks DESC
    

 * sqlite:///chinook.db
Done.


genre,number_of_tracks,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


In [32]:

query = """WITH 
    tracks_usa AS
    (
        SELECT 
        t.track_id track_id,
        g.name genre,
        c.country country   

        FROM invoice i
        LEFT JOIN customer c ON c.customer_id = i.customer_id
        LEFT JOIN invoice_line il ON il.invoice_id = i.invoice_id
        LEFT JOIN track t ON il.track_id = t.track_id
        LEFT JOIN genre g ON t.genre_id = g.genre_id
        WHERE country = "USA"
    )
    
SELECT
    tu.genre genre,
    COUNT(tu.track_id) number_of_tracks,
    CAST(COUNT(tu.track_id) AS FLOAT) / (SELECT COUNT(*) 
                                             FROM tracks_usa) AS percentage_sold
    FROM tracks_usa tu
    GROUP BY genre ORDER BY number_of_tracks DESC
    """

# Make connection to database
data = sqlite3.connect('chinook.db')

# Convert the query output into a dataframe
genre_pop = pd.read_sql_query(query, data)
genre_pop = genre_pop.append(genre_pop.loc[9:, ['number_of_tracks','percentage_sold']].sum(),ignore_index=True)
genre_pop.iloc[-1, 0] = 'Other'
genre_pop.drop(index=list(range(9, 17)), inplace=True)

# Create a pie plot of the results
fig = px.pie(genre_pop, names='genre', values='number_of_tracks', 
            title='Genre Popularity in US Market', 
            color_discrete_sequence=px.colors.sequential.RdBu_r)
fig.update_layout(legend_title_text='Genre')
fig.show()

## Based on the results Punk, Blues and Pop would be the three recommended genres.

# High Performing Employees in the US


In [38]:
%%sql
WITH
    query_1 AS
    (
        SELECT 
        e.employee_id employee_id,
        e.first_name || " " || e.last_name employee_name,
        e.hire_date hire_date,
        SUM(i.total) total_sales
        
        FROM customer c
        INNER JOIN employee e ON c.support_rep_id = e.employee_id
        INNER JOIN invoice i ON i.customer_id = c.customer_id
        WHERE e.title= "Sales Support Agent"
        GROUP BY employee_id
    )
SELECT * FROM query_1
    
    

 * sqlite:///chinook.db
Done.


employee_id,employee_name,hire_date,total_sales
3,Jane Peacock,2017-04-01 00:00:00,1731.5100000000039
4,Margaret Park,2017-05-03 00:00:00,1584.0000000000034
5,Steve Johnson,2017-10-17 00:00:00,1393.920000000002


In [45]:


query = """WITH 
    query_1 AS
    (
        SELECT 
        e.employee_id employee_id,
        e.first_name || " " || e.last_name employee_name,
        e.hire_date hire_date,
        SUM(i.total) total_sales
        
        FROM customer c
        INNER JOIN employee e ON c.support_rep_id = e.employee_id
        INNER JOIN invoice i ON i.customer_id = c.customer_id
        WHERE e.title= "Sales Support Agent"
        GROUP BY employee_id
    )
SELECT * FROM query_1"""


employee_sales = pd.read_sql_query(query , data)

fig = px.bar(employee_sales, x='employee_name', y='total_sales', text='total_sales', 
            title='Chinook: Total Sales by Agents', 
            hover_data=['employee_id', 'total_sales', 'hire_date'])
fig.update_xaxes(title_text='Employee_Name')
fig.update_yaxes(title_text='Total_Sales', showticklabels=False)
fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
fig.update_traces(marker_color='rgb(207, 207, 207)')
fig.show()

### While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.


## Sales by Country
It would be interesting and informative to find out which country provides the most business for Chinook. To do this, a number of different statistics will be calculated for each country. 

The statistics for these 24 countries will be displayed in a table with the following fields: 

**Field Name** | **Description**
:---: | :---
Number of Customers | The number of customers 
Total Sales Amount | The total amount spent by customers 
Average Sale Amount | The average amount per purchase by customers 
Average Customer Value | The average amount that customers have spent to date

It is important to note that some countries have more than one customer, while others have only one. These latter countries are grouped into 'Other' in the results. 

In [47]:
%%sql

WITH 
    country_or_other AS
    (
        SELECT 
            CASE
                WHEN (
                SELECT COUNT(*) FROM customer
                    WHERE country = c.country           
                ) = 1 THEN "Other"
                ELSE c.country
            END AS country,
            c.customer_id,
            il.*
            FROM invoice_line il
            INNER JOIN invoice i ON i.invoice_id = il.invoice_id
            INNER JOIN customer c ON c.customer_id = i.customer_id        
    )
SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );
    

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


In [51]:
query = """WITH 
    country_or_other AS
    (
        SELECT 
            CASE
                WHEN (
                SELECT COUNT(*) FROM customer
                    WHERE country = c.country           
                ) = 1 THEN "Other"
                ELSE c.country
            END AS country,
            c.customer_id,
            il.*
            FROM invoice_line il
            INNER JOIN invoice i ON i.invoice_id = il.invoice_id
            INNER JOIN customer c ON c.customer_id = i.customer_id        
    )
SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );
    """

country_data = pd.read_sql_query(query, data)

# Create bar plot of data
fig = px.bar(country_data, x='total_sales', y='country', title='Sales by Country', 
            hover_data=country_data.columns[1:], 
            text='total_sales', 
            orientation='h')
fig.update_xaxes(tickangle=90, showticklabels=False)
fig.update_traces(marker_color='rgb(207, 207, 207)')
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.show()


In [54]:
fig = px.scatter_matrix(country_data.iloc[:, 1:])
fig.update_layout(height=700, width=970, title='Country Statistics Correlation', title_x=.5)
fig.update_traces(showupperhalf=False, diagonal_visible=False)
fig.show()

Based on the data, there may be opportunity in the following countries:

Czech Republic
United Kingdom
India
It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Albums vs Individual Tracks?



Chinook management is deliberating whether they should purchase individual, popular tracks rather than whole albums from record companies. To aid in this decision, we have to figure out if the majority of customer purchases are individual tracks or whole albums. 

Approach: 
1. Check each invoice. 
2. Select all tracks associated with that invoice.
2. Find the album associated with any one of those tracks. 
3. Compare the tracks from the album to those of the invoice. 
4. If the same tracks and number of tracks are in both, then this is an album purchase. 

In [53]:
%%sql
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


The above query finds the track_id with the lowest number in the invoice_line table for each invoice_id. However since an invoice can either be an album purchase or individual track purchases, we cannot say that every track_id we have is the first track of the album. It is only first, if the invoice was an album purchase. But this track_id will help us to identify whether it was an album purchase or individual track purchases. And we can do so by getting all the tracks in an album which contains our first track_id and comparing it with all the tracks in the invoice, if both are returning the same set of tracks then it is an album purchase, otherwise it is individual track purchases. 

## Recommendation: The Chinook store should not continue to make album purchases as 81% of the purchases are track purchases.